# Load packages

In [ ]:
from environment import Santa2022Environment
from utils import *

import pandas as pd
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import CheckpointCallback
from gym.envs.registration import register

import matplotlib.pyplot as plt

# Register Env

In [ ]:
register(
    id='kaggle_santa/Santa2022-v0',
    entry_point='environment:Santa2022Environment',
    max_episode_steps=1e09
)

# Load Image of Christmas card

In [ ]:
df_image = pd.read_csv("image.csv")
image = df_to_image(df_image)

In [ ]:
plt.imshow(image)
plt.show()

# Load Gym Env

In [ ]:
# Parallel environments
env = make_vec_env("kaggle_santa/Santa2022-v0", n_envs=4, env_kwargs={"image": image})

# Create instance of PPO model

In [ ]:
model = PPO(
    "MlpPolicy",
    env = env,
    verbose=1
)
checkpoint_callback = CheckpointCallback(
  save_freq=10000,
  save_path="models/",
  name_prefix="rl_model",
)

# Run Training

In [ ]:
model.learn(total_timesteps=int(3e8), callback=checkpoint_callback)